1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
def bin_to_dec(n):
    return int(n, 2)

def dec_to_bin(n):
    return bin(n)

def hex_to_bin(n):
    return bin(int(n,16))

def bin_to_hex(n):
    return hex(int(n,2))

def dec_to_hex(n):
    return hex(n)

def hex_to_dec(n):
    return int(n, 16)

n = 20

bin_n = bin(n)
hex_n = hex(n)

print("the number in base 10 is :",n)
print("base 10->2:","\t",n,"->",dec_to_bin(n))
print("base 10->16:","\t",n,"->",dec_to_hex(n))
print("base 10->16:","\t",n,"->",dec_to_hex(n))
print("base 2->10: ","\t",bin_n,"->",bin_to_dec(bin_n))
print("base 16->10:","\t",hex_n,"-> ",hex_to_dec(hex_n))
print("base 2->16:","\t",bin_n,"->",bin_to_hex(bin_n))
print("base 16->2:","\t",hex_n,"->",hex_to_bin(hex_n))



the number in base 10 is : 20
base 10->2: 	 20 -> 0b10100
base 10->16: 	 20 -> 0x14
base 10->16: 	 20 -> 0x14
base 2->10:  	 0b10100 -> 20
base 16->10: 	 0x14 ->  20
base 2->16: 	 0b10100 -> 0x14
base 16->2: 	 0x14 -> 0b10100


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [4]:
def convert(word):
    #sign
    sign = int(word[31])
    
    #exponent
    exp = word[23:31]
    exp = reversed(exp)
    exp = "".join([str(item) for item in exp])
    exp_dec = int(exp,2)
    
    #mantissa
    mant = word[0:23]
    mant = list(mant)
    mant.reverse()
    mant_dec = 0
    for i in range(len(mant)):
        temp = int(mant[i])
        if temp == 1:
            mant_dec += pow(2,-(i+1))
    n = (mant_dec+1)*pow(2,exp_dec-127)
    
    
    result = 0
    if sign == 0:
        result = n
    else:
        result = -n
        
    return result

word = "00000000000000000000000010000001"
print(convert(word))


-2.350988701644575e-38


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [5]:
import math

low = 1.0
height = 1.0
zero = 0.0
inf = math.inf
iter = 0

#the last number is 4.94066e-324
while(low.hex() != zero.hex()):
    low = low/2
    iter += 1
print("underflow in iteration",iter,"\t\t","%2.5e"%low,)

#the last number is 8.98847e+307
iter = 0
while(height.hex() != inf.hex()):
    height = height*2
    iter += 1
print("overflow in iteration",iter,"\t\t","%2.5e"%height)


underflow in iteration 1075 		 0.00000e+00
overflow in iteration 1024 		 inf


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [6]:
N = 1500
i = 0
number = 60.0
small = 1
flag = True

while (i in range(N) and flag == True):
        sum = number + small
        small = small/2 #smaller value
        
        if(sum.hex() == number.hex()): #precision found
            flag = False
            print("sum has no more effect. Iteration",i)
            
        #print(sum,"\t,\t",small)
        i += 1

print("the machine precision is", small)

sum has no more effect. Iteration 48
the machine precision is 1.7763568394002505e-15


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [7]:
import math

def quadratic_eq_1(a, b, c):
    x1 = (-b+math.sqrt(b**2-4*a*c))/(2*a)
    x2 = (-b-math.sqrt(b**2-4*a*c))/(2*a)
    return (x1,x2)

def quadratic_eq_2(a, b, c):
    mul_x1 = -b-math.sqrt(b**2-4*a*c)
    mul_x2 = -b+math.sqrt(b**2-4*a*c)

    x1 = ((-b+math.sqrt(b**2-4*a*c))*mul_x1)/((2*a)*mul_x1)
    x2 = ((-b-math.sqrt(b**2-4*a*c))*mul_x2)/((2*a)*mul_x2)
    return (x1,x2)

def quadratic_eq_corr(a, b, c):
    delta = b**2-4*a*c
    if delta > 0: 
        #Real and different roots
        x1 = (-b+math.sqrt(delta))/(2*a)
        x2 = (-b-math.sqrt(delta))/(2*a)
        return(x1, x2)
      
    elif delta == 0: 
        #Real and same roots
        x1 = x2 = -b/(2*a)
        return(x1, x2)
      
        #discriminant < 0
    else:
        print("\nComplex Roots") 
        x1 = -b/(2*a), " + i", sqrt_val
        x2 = -b/(2*a), " - i", sqrt_val
        return(x1, x2)

    
a = 0.001
b = 1000
c = 0.001

r1 = quadratic_eq_1(a,b,c)
r2 = quadratic_eq_2(a,b,c)
r3 = quadratic_eq_corr(a,b,c)
print("quadratic_eq_1 =",r1)
print("quadratic_eq_2 =",r2,"\n")
print("The results of quadratic_eq_1 and of quadratic_eq_2 are different. The reason is that float have limited precision.\n")

print("quadratic_eq_corr =",r3)



quadratic_eq_1 = (-9.999894245993346e-07, -999999.999999)
quadratic_eq_2 = (-9.999894245993346e-07, -999999.9999990001) 

The results of quadratic_eq_1 and of quadratic_eq_2 are different. The reason is that float have limited precision.

quadratic_eq_corr = (-9.999894245993346e-07, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [8]:
def function(x):
    return x*(x-1)

def incremental_ratio(delta):
    return (function(x+delta)-function(x))/delta
    
x = 1

for i in range(-2,-16,-2):
    delta = pow(10,i)
    print("delta= ",delta,"\t",incremental_ratio(delta))


print("\n")
print("Accuracy increases as delta decreases. However a very small delta leads to an error in the calculation of the derivative")

delta=  0.01 	 1.010000000000001
delta=  0.0001 	 1.0000999999998899
delta=  1e-06 	 1.0000009999177333
delta=  1e-08 	 1.0000000039225287
delta=  1e-10 	 1.000000082840371
delta=  1e-12 	 1.0000889005833413
delta=  1e-14 	 0.9992007221626509


Accuracy increases as delta decreases. However a very small delta leads to an error in the calculation of the derivative


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [1]:
import math
import time

#a)

N1=100
h = 2/N1
I = 0

for k in range(1, N1+1):
    y = math.sqrt(1-(h*k-1)**2)
    I = I + abs(y*h)
print("For N=100, I =",I)


#b)
#1 sec
k = 0
N2 = 1500
h2 = 2/N2
I2 = 0

start = time.time()

for k in range(1, N2):
    yk = math.sqrt(1-(h2*k-1)**2)
    I2 = I2+abs(h2*yk)
end = time.time()


print("Runtime is",end - start)
print("For N=", k, "I =", I2)
print("The gain in running 1s is:", I2-I)

#60 sec
k = 0
N3 = 115540000
h3 = 2/N3
I3 = 0

start = time.time()

for k in range(1, N3):
    yk = math.sqrt(1-(h3*k-1)**2)
    I3 = I3+abs(h3*yk)
end = time.time()

#60 sec
print("Runtime is",end - start)
print("For N=", k, "I =", I3)
print("The gain in running 60s is:", I3-I)


For N=100, I = 1.5691342555492505
Runtime is 0.0016570091247558594
For N= 1499 I = 1.5707677007686625
The gain in running 1s is: 0.0016334452194119997
Runtime is 58.73915386199951
For N= 115539999 I = 1.5707963267939176
The gain in running 60s is: 0.001662071244667107
